In [1]:
from google.colab import drive
import os
import shutil
import pandas as pd
import random

# Montar Google Drive
drive.mount('/content/drive')

# Ajustar caminhos para o Drive
RAW_PATH = "/content/drive/MyDrive/Colab_Datasets/skin-cancer-mnist-ham10000"  # Ajuste conforme sua estrutura
OUT_PATH = "/content/dataset/ham10000"

IMG_DIRS = [
    "HAM10000_images_part_1",
    "HAM10000_images_part_2"
]

META = os.path.join(RAW_PATH, "HAM10000_metadata.csv")

# Verificar se existe
print(f"Arquivo existe: {os.path.exists(META)}")

# parâmetros few-shot
N_CLASSES = 5
N_SHOTS = 5
N_TEST = 15

random.seed(42)

# lê metadata
df = pd.read_csv(META)
# seleciona classes mais comuns (evita classes raras)
classes = df['dx'].value_counts().index[:N_CLASSES]

print("Classes usadas:", classes.tolist())

# cria pastas
for split in ['train', 'test']:
    for c in classes:
        os.makedirs(f"{OUT_PATH}/{split}/{c}", exist_ok=True)

# separação
for c in classes:
    imgs = df[df['dx'] == c]['image_id'].tolist()
    random.shuffle(imgs)

    train_imgs = imgs[:N_SHOTS]
    test_imgs  = imgs[N_SHOTS:N_SHOTS + N_TEST]

    for img in train_imgs:
        for d in IMG_DIRS:
            src = os.path.join(RAW_PATH, d, img + ".jpg")
            if os.path.exists(src):
                shutil.copy(src, f"{OUT_PATH}/train/{c}/")

    for img in test_imgs:
        for d in IMG_DIRS:
            src = os.path.join(RAW_PATH, d, img + ".jpg")
            if os.path.exists(src):
                shutil.copy(src, f"{OUT_PATH}/test/{c}/")

print("Separação concluída!")

Mounted at /content/drive
Arquivo existe: True
Classes usadas: ['nv', 'mel', 'bkl', 'bcc', 'akiec']
Separação concluída!


Modelo

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader
from sklearn.metrics import balanced_accuracy_score, classification_report
from PIL import Image

cudnn.deterministic = False
cudnn.benchmark = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [3]:


class LinearClassifier(nn.Module):
    def __init__(self, in_features, num_classes):
        super().__init__()
        # O baseline supervisionado usa uma projeção linear direta para as classes
        self.fc = nn.Linear(in_features, num_classes)

    def forward(self, x):
        return self.fc(x)

In [4]:


transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [5]:


train_dataset = datasets.ImageFolder(
    root="/content/dataset/ham10000/train",
    transform=transform_train
)

test_dataset = datasets.ImageFolder(
    root="/content/dataset/ham10000/test",
    transform=transform_test
)

train_loader = DataLoader(
    train_dataset,
    batch_size=32, # Batch size padrão (aumentei para 32 para estabilidade do SGD)
    shuffle=True,
    num_workers=2
)

test_loader = DataLoader(
    test_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=2
)

classes = train_dataset.classes
num_classes = len(classes)

In [6]:

# RegNet (O artigo usa ViT/ResNet, mas a lógica de Fine-tuning é a mesma)
model = models.regnet_y_3_2gf(
    weights=models.RegNet_Y_3_2GF_Weights.IMAGENET1K_V2
)

in_features = model.fc.in_features

# Substituindo a cabeça pela Linear Layer padrão
model.fc = nn.Linear(in_features, num_classes)

model = model.to(device)

Downloading: "https://download.pytorch.org/models/regnet_y_3_2gf-9180c971.pth" to /root/.cache/torch/hub/checkpoints/regnet_y_3_2gf-9180c971.pth


100%|██████████| 74.6M/74.6M [00:00<00:00, 190MB/s]


In [8]:
#função de perda
criterion = nn.CrossEntropyLoss(
    label_smoothing=0.05
)

In [9]:
# Configuração para treinamento supervisionado
EPOCHS = 10

for p in model.parameters():
    p.requires_grad = True


# O artigo usa SGD com momentum 0.9
# LR maior na cabeça pq ela é nova
optimizer = optim.SGD([
    {'params': model.trunk_output.parameters(), 'lr': 1e-5}, # Backbone (LR baixa do artigo)
    {'params': model.fc.parameters(), 'lr': 1e-2}            # Cabeça (LR maior para aprender rápido)
], momentum=0.9, weight_decay=1e-4)

# Scheduler Cosseno [cite: 195]
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)

In [12]:
model.train()

for epoch in range(EPOCHS):
    running_loss = 0.0
    correct = 0
    total = 0

    # treinamento supervisionado
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        # Forward
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward
        loss.backward()
        optimizer.step()

        # Métricas
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # Atualiza o scheduler ao fim da época
    scheduler.step()

    epoch_acc = 100 * correct / total
    avg_loss = running_loss / len(train_loader)

    print(f"Epoch [{epoch+1}/{EPOCHS}] Loss: {avg_loss:.4f} Acc: {epoch_acc:.2f}%")

Epoch [1/10] Loss: 1.6077 Acc: 40.00%
Epoch [2/10] Loss: 1.4031 Acc: 44.00%
Epoch [3/10] Loss: 1.2593 Acc: 40.00%
Epoch [4/10] Loss: 1.0914 Acc: 48.00%
Epoch [5/10] Loss: 0.9258 Acc: 64.00%
Epoch [6/10] Loss: 0.8100 Acc: 76.00%
Epoch [7/10] Loss: 0.8816 Acc: 68.00%
Epoch [8/10] Loss: 0.7810 Acc: 84.00%
Epoch [9/10] Loss: 0.7346 Acc: 80.00%
Epoch [10/10] Loss: 0.8291 Acc: 76.00%


In [13]:
from sklearn.metrics import classification_report, balanced_accuracy_score

# "perform 50 adaptation steps for tasks from novel classes during evaluation phase"

def test_with_adaptation(model, test_loader, n_adaptation_steps=50):
    model.eval()

    # congela o backbone (pra não destruir o conhecimento base)
    for param in model.parameters():
        param.requires_grad = False

    # libera apenas a cabeça para treino
    for param in model.fc.parameters():
        param.requires_grad = True

    # otimizador
    adapt_optimizer = optim.SGD(model.fc.parameters(), lr=0.01, momentum=0.9)

    print(f"Iniciando {n_adaptation_steps} passos de adaptação no teste")

    # pega um batch do teste para treinar 'K-shots'
    iter_test = iter(test_loader)
    support_images, support_labels = next(iter_test) # pega 1 batch para adaptar
    support_images, support_labels = support_images.to(device), support_labels.to(device)

    model.train()

    for i in range(n_adaptation_steps):
        adapt_optimizer.zero_grad()
        outputs = model(support_images)
        loss = criterion(outputs, support_labels)
        loss.backward()
        adapt_optimizer.step()

    print("Adaptação concluída. Avaliando no restante")

    # avaliação real (nas imagens restantes)
    model.eval()
    all_preds = []
    all_true = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_true.extend(labels.cpu().numpy())

    return all_true, all_preds


# chamada do teste

print("Iniciando avaliação com adaptação (Baseline do Artigo)")

# roda a adaptação e pega as predições
y_true, y_pred = test_with_adaptation(model, test_loader, n_adaptation_steps=50)

# gera as métricas
print(f"\nBalanced Accuracy: {balanced_accuracy_score(y_true, y_pred)*100:.2f}%")
print(classification_report(y_true, y_pred, target_names=classes))

Iniciando avaliação com adaptação (Baseline do Artigo)...
Iniciando 50 passos de adaptação no teste
Adaptação concluída. Avaliando no restante

Balanced Accuracy: 49.33%
              precision    recall  f1-score   support

       akiec       0.47      1.00      0.64        15
         bcc       0.48      1.00      0.65        15
         bkl       0.55      0.40      0.46        15
         mel       0.00      0.00      0.00        15
          nv       1.00      0.07      0.12        15

    accuracy                           0.49        75
   macro avg       0.50      0.49      0.38        75
weighted avg       0.50      0.49      0.38        75



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
